<a href="https://colab.research.google.com/github/zealpatel1990/KDM_spring_2021_class/blob/main/ICP%205/source_code/KDM_ICP5_zeal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     |████████████████████████████████| 204.8MB 33kB/s 
     |████████████████████████████████| 204kB 39.4MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.2-py2.py3-none-any.whl size=205186687 sha256=643e682b9677b01095c7d19cda29a053159faab6acf2372eb21f0525baf161b2
  Stored in directory: /root/.cache/pip/wheels/8b/09/da/c1f2859bcc86375dc972c5b6af4881b3603269bcc4c9be5d16
Successfully built pyspark


In [2]:
from __future__ import print_function
from pyspark import SparkConf, SparkContext
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.sql import SparkSession
from pyspark.ml.feature import NGram
from pyspark.ml.feature import Word2Vec

In [3]:
# creating spark session
spark = SparkSession.builder.appName("TfIdf Example").getOrCreate()

In [4]:
# creating spark dataframe wiht the input data. You can also read the data from file. label represents the 3 documnets (0.0,0.1,0.2)
sentenceData = spark.createDataFrame([
        (0.0, "Welcome to KDM TF_IDF Tutorial."),
        (0.1, "Learn Spark ml tf_idf in today's lab."),
        (0.2, "Spark Mllib has TF-IDF.")
    ], ["label", "sentence"])

In [5]:
# creating tokens/words from the sentence data
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
wordsData = tokenizer.transform(sentenceData)

In [6]:
wordsData.show()

+-----+--------------------+--------------------+
|label|            sentence|               words|
+-----+--------------------+--------------------+
|  0.0|Welcome to KDM TF...|[welcome, to, kdm...|
|  0.1|Learn Spark ml tf...|[learn, spark, ml...|
|  0.2|Spark Mllib has T...|[spark, mllib, ha...|
+-----+--------------------+--------------------+



In [7]:
# applying tf on the words data
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=20)
featurizedData = hashingTF.transform(wordsData)
# alternatively, CountVectorizer can also be used to get term frequency vectors

In [8]:
# calculating the IDF
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

In [9]:
#displaying the results
rescaledData.select("label", "features").show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(20,[2,8,13,15,17...|
|  0.1|(20,[2,3,6,7],[0....|
|  0.2|(20,[6,14,15],[0....|
+-----+--------------------+



In [10]:
spark2 = SparkSession.builder.appName("Ngram Example").getOrCreate()

In [11]:
#creating dataframe of input
wordDataFrame = spark2.createDataFrame([
    (0, ["Hi", "I", "heard", "about", "Spark"]),
    (1, ["I", "wish", "Java", "could", "use", "case", "classes"]),
    (2, ["Logistic", "regression", "models", "are", "neat"])
], ["id", "words"])


In [12]:
#creating NGrams with n=2 (two words)
ngram = NGram(n=2, inputCol="words", outputCol="ngrams")
ngramDataFrame = ngram.transform(wordDataFrame)

In [13]:
# displaying the results
ngramDataFrame.select("ngrams").show(truncate=False)

+------------------------------------------------------------------+
|ngrams                                                            |
+------------------------------------------------------------------+
|[Hi I, I heard, heard about, about Spark]                         |
|[I wish, wish Java, Java could, could use, use case, case classes]|
|[Logistic regression, regression models, models are, are neat]    |
+------------------------------------------------------------------+



In [14]:
# creating spark session
spark3 = SparkSession.builder.appName("Word2Vec Example").getOrCreate()

In [15]:
# Input data: Each row is a bag of words from a sentence or document.
documentDF = spark3.createDataFrame([
    ("McCarthy was asked to analyse the data from the first phase of trials of the vaccine.".split(" "), ),
    ("We have amassed the raw data and are about to begin analysing it.".split(" "), ),
    ("Without more data we cannot make a meaningful comparison of the two systems.".split(" "), ),
    ("Collecting data is a painfully slow process.".split(" "), ),
    ("You need a long series of data to be able to discern such a trend.".split(" "), )
], ["text"])

In [16]:
# Learn a mapping from words to Vectors.
word2Vec = Word2Vec(vectorSize=3, minCount=0, inputCol="text", outputCol="result")
model = word2Vec.fit(documentDF)
result = model.transform(documentDF)

In [17]:
for row in result.collect():
    text, vector = row
    #printing the results
    print("Text: [%s] => \nVector: %s\n" % (", ".join(text), str(vector)))

Text: [McCarthy, was, asked, to, analyse, the, data, from, the, first, phase, of, trials, of, the, vaccine.] => 
Vector: [0.008847468649037182,0.027308731980156153,0.005336133428500034]

Text: [We, have, amassed, the, raw, data, and, are, about, to, begin, analysing, it.] => 
Vector: [0.01598473423375533,0.024079730084989794,0.022829564359898753]

Text: [Without, more, data, we, cannot, make, a, meaningful, comparison, of, the, two, systems.] => 
Vector: [0.008413167097247565,-0.0008237379101606517,-0.04820680912010945]

Text: [Collecting, data, is, a, painfully, slow, process.] => 
Vector: [-0.020028315750615935,0.03329797900680984,0.01771639713219234]

Text: [You, need, a, long, series, of, data, to, be, able, to, discern, such, a, trend.] => 
Vector: [0.007570979123314222,0.00809092471996943,0.003313163419564565]



In [18]:
# showing the synonyms and cosine similarity of the word in input data
synonyms = model.findSynonyms("data", 5)   # its okay for certain words , real bad for others
synonyms.show(5)

+-----+------------------+
| word|        similarity|
+-----+------------------+
|  raw|0.9751740097999573|
|first|0.9728763699531555|
|   We|0.8854662775993347|
|   is|0.7463950514793396|
|  it.|0.7444145679473877|
+-----+------------------+



In [19]:
#closing the spark sessions
spark.stop()
spark2.stop()
spark3.stop()

# Task 1:

In [32]:
with open("/content/t1.txt","r+") as t1:
    doc1 = t1.read()
with open("/content/t2.txt","r+") as t2:
    doc2 = t2.read()
with open("/content/t3.txt","r+") as t3:
    doc3 = t3.read()
with open("/content/t4.txt","r+") as t4:
    doc4 = t4.read()
with open("/content/t5.txt","r+") as t5:
    doc5 = t5.read()
# Read all 5 txt files which contains news articles
documents = [doc1,doc2,doc3,doc4,doc5]

## a. Top 10 TF-IDF words for the above input

In [107]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# using sklearn library which has inbuilt Tfidf vectorizer class which can generate tfidf for given corpus
vect = TfidfVectorizer()
#created TfidfVectorizer object
tfidf_matrix = vect.fit_transform(documents)
#passed list of documents or corpus to obt method fit_transform
df = pd.DataFrame(tfidf_matrix.toarray(), columns = vect.get_feature_names())
# converted method output to panda data frame 
pd.set_option('display.max_columns', 20)

df.loc['Total'] = df.sum() # adding row to value total

#filtering values of words whos tfidf is greater than 0.3
# also used transpose function here to filter out words (which was rows) and then converted matrix back to original version
print (df.T.sort_values('Total', ascending=True).tail(10).T)

           said        on   bitcoin       for      that        in        to  \
0      0.044284  0.094525  0.000000  0.037253  0.088568  0.094525  0.157541   
1      0.139950  0.033192  0.000000  0.078487  0.233249  0.066384  0.099575   
2      0.124685  0.044357  0.000000  0.000000  0.000000  0.088714  0.044357   
3      0.000000  0.078588  0.000000  0.139374  0.000000  0.196469  0.157175   
4      0.000000  0.067475  0.354008  0.119665  0.094833  0.033737  0.168687   
Total  0.308918  0.318136  0.354008  0.374779  0.416650  0.479829  0.627336   

             of       and       the  
0      0.157541  0.094525  0.535641  
1      0.165959  0.232342  0.265534  
2      0.177429  0.133071  0.443572  
3      0.078588  0.196469  0.392938  
4      0.202424  0.134950  0.236162  
Total  0.781941  0.791357  1.873846  


## b. Top 10 TF-IDF words for the lemmatized input

In [109]:
import nltk;nltk.download('punkt');nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

words1 = nltk.word_tokenize(doc1)
words2 = nltk.word_tokenize(doc2)
words3 = nltk.word_tokenize(doc3)
words4 = nltk.word_tokenize(doc4)
words5 = nltk.word_tokenize(doc5)

lemmatized_document1 = ' '.join([lemmatizer.lemmatize(w) for w in words1])
lemmatized_document2 = ' '.join([lemmatizer.lemmatize(w) for w in words2])
lemmatized_document3 = ' '.join([lemmatizer.lemmatize(w) for w in words3])
lemmatized_document4 = ' '.join([lemmatizer.lemmatize(w) for w in words4])
lemmatized_document5 = ' '.join([lemmatizer.lemmatize(w) for w in words5])

documents = [lemmatized_document1,lemmatized_document2,lemmatized_document3,lemmatized_document4,lemmatized_document5]

# using sklearn library which has inbuilt Tfidf vectorizer class which can generate tfidf for given corpus
vect = TfidfVectorizer()
#created TfidfVectorizer object
tfidf_matrix = vect.fit_transform(documents)
#passed list of documents or corpus to obt method fit_transform
df = pd.DataFrame(tfidf_matrix.toarray(), columns = vect.get_feature_names())
# converted method output to panda data frame 

df.loc['Total'] = df.sum() # adding row to value total

#filtering values of words whos tfidf is greater than 0.3
# also used transpose function here to filter out words (which was rows) and then converted matrix back to original version
print (df.T.sort_values('Total', ascending=True).tail(10).T)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
           said        on   bitcoin       for      that        in        to  \
0      0.044284  0.094525  0.000000  0.037253  0.088568  0.094525  0.157541   
1      0.139950  0.033192  0.000000  0.078487  0.233249  0.066384  0.099575   
2      0.124685  0.044357  0.000000  0.000000  0.000000  0.088714  0.044357   
3      0.000000  0.078588  0.000000  0.139374  0.000000  0.196469  0.157175   
4      0.000000  0.067475  0.354008  0.119665  0.094833  0.033737  0.168687   
Total  0.308918  0.318136  0.354008  0.374779  0.416650  0.479829  0.627336   

             of       and       the  
0      0.157541  0.094525  0.535641  
1      0.165959  0.232342  0.265534  
2      0.177429  0.133071  0.443572  
3      0.078588  0.196469  0.392938  
4      0.202424  0.134950  0.2

## c. Top 10 TF-IDF words for the n-gram based input.

In [104]:
# this function takes document and n int value to generate list of n grams
def ngrams(input, n):
    input = input.split(' ')
    output = []
    for i in range(len(input)-n+1):
        output.append(input[i:i+n])
    return output

ngram_doc1 = ' '.join([' '.join(x) for x in ngrams(doc1, 3)])
ngram_doc2 = ' '.join([' '.join(x) for x in ngrams(doc2, 3)])
ngram_doc3 = ' '.join([' '.join(x) for x in ngrams(doc3, 3)])
ngram_doc4 = ' '.join([' '.join(x) for x in ngrams(doc4, 3)])
ngram_doc5 = ' '.join([' '.join(x) for x in ngrams(doc5, 3)])

# documents = [ngram_doc1,ngram_doc2,ngram_doc3,ngram_doc4,ngram_doc5]

documents = [doc1,doc2,doc3,doc4,doc5]

# using sklearn library which has inbuilt Tfidf vectorizer class which can generate tfidf for given corpus
vect = TfidfVectorizer( ngram_range=(3,3)) # TfidfVectorizer has inbuilt ngram kwarg which show tfidf for ngrams
#created TfidfVectorizer object
tfidf_matrix = vect.fit_transform(documents)
#passed list of documents or corpus to obt method fit_transform
df = pd.DataFrame(tfidf_matrix.toarray(), columns = vect.get_feature_names())
# converted method output to panda data frame 

df.loc['Total'] = df.sum() # adding row to value total

#filtering values of words whos tfidf is greater than 0.3
# also used transpose function here to filter out words (which was rows) and then converted matrix back to original version
print (df.T.sort_values('Total', ascending=True).tail(10).T)

       judicial review proceedings  kingdom as named  \
0                         0.000000          0.000000   
1                         0.000000          0.000000   
2                         0.105409          0.105409   
3                         0.000000          0.000000   
4                         0.000000          0.000000   
Total                     0.105409          0.105409   

       joining other likeminded  as well as  by the end  the end of  \
0                      0.000000    0.000000     0.15861     0.15861   
1                      0.000000    0.064461     0.00000     0.00000   
2                      0.105409    0.000000     0.00000     0.00000   
3                      0.000000    0.073451     0.00000     0.00000   
4                      0.000000    0.000000     0.00000     0.00000   
Total                  0.105409    0.137912     0.15861     0.15861   

       the united states  republican accountability project  \
0                0.15861                      

# Task 2:


In [148]:
from __future__ import print_function
from pyspark import SparkConf, SparkContext
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.sql import SparkSession
from pyspark.ml.feature import NGram
from pyspark.ml.feature import Word2Vec
# creating spark session
spark = SparkSession.builder.appName("TfIdf Example").getOrCreate()

documentData = spark.createDataFrame([
        (0.0, doc1),
        (0.1, doc2),
        (0.2, doc3),
        (0.3, doc4),
        (0.5, doc5)
    ], ["label", "document"])

# creating tokens/words from the sentence data
tokenizer = Tokenizer(inputCol="document", outputCol="words")
wordsData = tokenizer.transform(documentData)
print (documentData)
wordsData.show()

DataFrame[label: double, document: string]
+-----+--------------------+--------------------+
|label|            document|               words|
+-----+--------------------+--------------------+
|  0.0|In his first trip...|[in, his, first, ...|
|  0.1|“It’s a really im...|[“it’s, a, really...|
|  0.2|The protocol was ...|[the, protocol, w...|
|  0.3|Copper prices cli...|[copper, prices, ...|
|  0.5|Many of the same ...|[many, of, the, s...|
+-----+--------------------+--------------------+



## a. Try without NLP

In [141]:
# applying tf on the words data
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=200)
tf = hashingTF.transform(wordsData)
# alternatively, CountVectorizer can also be used to get term frequency vectors
# calculating the IDF
tf.cache()
idf = IDF(inputCol="rawFeatures", outputCol="features")
idf = idf.fit(tf)
tfidf = idf.transform(tf)
#displaying the results
tfidf.select("label", "features").show()


print("TF-IDF without NLP:")
for each in tfidf.collect():
    print(each)
    print(each['rawFeatures'])
spark.stop()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(200,[1,3,5,7,9,1...|
|  0.1|(200,[0,4,5,7,9,1...|
|  0.2|(200,[7,8,17,20,2...|
|  0.3|(200,[2,3,5,9,12,...|
|  0.5|(200,[1,2,3,4,5,7...|
+-----+--------------------+

TF-IDF without NLP:
Row(label=0.0, document='In his first trips as president, President Biden traveled to Wisconsin and Michigan to promote his vaccination rollout plan and the $1.9 trillion relief bill he hopes can restore the American economy.\n\nAfter an optimistic vow on Tuesday that any American who wanted a vaccine “could have one by the end of July this year,” Mr. Biden was asking for patience on Friday, saying the United States could be“approaching normalcy” by the end of the year. The week ended as winter storms hitting much of the country delayed the delivery of six million vaccines.\n\nMr. Biden addressed the virtual G7 summit and said that his administration would make good on a U.S. promise to donate $4 billion to t

## b. Try with Lemmatization

In [145]:
import nltk;nltk.download('punkt');nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

words1 = nltk.word_tokenize(doc1)
words2 = nltk.word_tokenize(doc2)
words3 = nltk.word_tokenize(doc3)
words4 = nltk.word_tokenize(doc4)
words5 = nltk.word_tokenize(doc5)

lemmatized_document1 = ' '.join([lemmatizer.lemmatize(w) for w in words1])
lemmatized_document2 = ' '.join([lemmatizer.lemmatize(w) for w in words2])
lemmatized_document3 = ' '.join([lemmatizer.lemmatize(w) for w in words3])
lemmatized_document4 = ' '.join([lemmatizer.lemmatize(w) for w in words4])
lemmatized_document5 = ' '.join([lemmatizer.lemmatize(w) for w in words5])

### lemmatizing words from 5 input docs same as previos task

# creating spark session
spark = SparkSession.builder.appName("TfIdf Example").getOrCreate()

documentData = spark.createDataFrame([
        (0.0, lemmatized_document1),
        (0.1, lemmatized_document2),
        (0.2, lemmatized_document3),
        (0.3, lemmatized_document4),
        (0.5, lemmatized_document5)
    ], ["label", "document"])

# creating tokens/words from the sentence data
tokenizer = Tokenizer(inputCol="document", outputCol="words")
wordsData = tokenizer.transform(documentData)
print (documentData)
wordsData.show()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
DataFrame[label: double, document: string]
+-----+--------------------+--------------------+
|label|            document|               words|
+-----+--------------------+--------------------+
|  0.0|In his first trip...|[in, his, first, ...|
|  0.1|“ It ’ s a really...|[“, it, ’, s, a, ...|
|  0.2|The protocol wa d...|[the, protocol, w...|
|  0.3|Copper price clim...|[copper, price, c...|
|  0.5|Many of the same ...|[many, of, the, s...|
+-----+--------------------+--------------------+



In [146]:
# applying tf on the words data
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=200)
tf = hashingTF.transform(wordsData)
# alternatively, CountVectorizer can also be used to get term frequency vectors
# calculating the IDF
tf.cache()
idf = IDF(inputCol="rawFeatures", outputCol="features")
idf = idf.fit(tf)
tfidf = idf.transform(tf)
#displaying the results
tfidf.select("label", "features").show()


print("TF-IDF with Lemmatization:")
for each in tfidf.collect():
    print(each)
    print(each['rawFeatures'])
spark.stop()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(200,[0,1,3,5,9,1...|
|  0.1|(200,[0,4,5,8,9,1...|
|  0.2|(200,[7,8,13,17,2...|
|  0.3|(200,[3,4,5,9,17,...|
|  0.5|(200,[1,2,3,4,5,6...|
+-----+--------------------+

TF-IDF without NLP:
Row(label=0.0, document='In his first trip a president , President Biden traveled to Wisconsin and Michigan to promote his vaccination rollout plan and the $ 1.9 trillion relief bill he hope can restore the American economy . After an optimistic vow on Tuesday that any American who wanted a vaccine “ could have one by the end of July this year , ” Mr. Biden wa asking for patience on Friday , saying the United States could be “ approaching normalcy ” by the end of the year . The week ended a winter storm hitting much of the country delayed the delivery of six million vaccine . Mr. Biden addressed the virtual G7 summit and said that his administration would make good on a U.S. promise to donate $ 4 billion to t

## c. Try with NGrams

In [156]:
spark = SparkSession.builder.appName("TfIdf Example").getOrCreate()

documentData = spark.createDataFrame([
        (0.0, doc1.split(' ')),
        (0.1, doc2.split(' ')),
        (0.2, doc3.split(' ')),
        (0.3, doc4.split(' ')),
        (0.5, doc5.split(' '))
    ], ["label", "document"])


ngram = NGram(n=2, inputCol="document", outputCol="ngrams")

ngramDataFrame = ngram.transform(documentData)

# applying tf on the words data
hashingTF = HashingTF(inputCol="ngrams", outputCol="rawFeatures", numFeatures=200)
tf = hashingTF.transform(ngramDataFrame)
# alternatively, CountVectorizer can also be used to get term frequency vectors
# calculating the IDF
tf.cache()
idf = IDF(inputCol="rawFeatures", outputCol="features")
idf = idf.fit(tf)
tfidf = idf.transform(tf)
#displaying the results
tfidf.select("label", "features").show()


print("TF-IDF with ngram:")
for each in tfidf.collect():
    print(each)
    print(each['rawFeatures'])
spark.stop()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(200,[6,7,11,15,1...|
|  0.1|(200,[0,3,5,6,11,...|
|  0.2|(200,[4,20,21,22,...|
|  0.3|(200,[0,1,2,3,4,6...|
|  0.5|(200,[0,1,2,4,5,6...|
+-----+--------------------+

TF-IDF without NLP:
Row(label=0.0, document=['In', 'his', 'first', 'trips', 'as', 'president,', 'President', 'Biden', 'traveled', 'to', 'Wisconsin', 'and', 'Michigan', 'to', 'promote', 'his', 'vaccination', 'rollout', 'plan', 'and', 'the', '$1.9', 'trillion', 'relief', 'bill', 'he', 'hopes', 'can', 'restore', 'the', 'American', 'economy.\n\nAfter', 'an', 'optimistic', 'vow', 'on', 'Tuesday', 'that', 'any', 'American', 'who', 'wanted', 'a', 'vaccine', '“could', 'have', 'one', 'by', 'the', 'end', 'of', 'July', 'this', 'year,”', 'Mr.', 'Biden', 'was', 'asking', 'for', 'patience', 'on', 'Friday,', 'saying', 'the', 'United', 'States', 'could', 'be“approaching', 'normalcy”', 'by', 'the', 'end', 'of', 'the', 'year.', 'The', 'week', 'en